# Environment Setting

In [ ]:
import aie

aie.Authenticate()
aie.Initialize()

work_dir = "./work_dirs/tutorial"

# Dataset

In [ ]:
# 下载样本集
from aiearth.deeplearning.cloud.datasets import LandcoverDataset, PublicDatasetMeta
gid_15_train_dataset = LandcoverDataset(PublicDatasetMeta.GID_15_TRAIN["dataset_id"], data_root=work_dir)

In [ ]:
from aiearth.deeplearning.sampler import RandomNonGeoDatasetSampler
# 随机按照80%， 20%进行切分成两个新样本集
train_dataset, val_dataset = RandomNonGeoDatasetSampler.split_by_percent(gid_15_train_dataset, 0.8)
# 随机提取20张图生成新样本集
test_dataset = RandomNonGeoDatasetSampler.sample_by_count(gid_15_train_dataset, 20) 

# Model

In [ ]:
# model
from aiearth.deeplearning.trainer.mmseg import LandcoverTrainer

trainer = LandcoverTrainer(work_dir=work_dir, config_name="fcn_hr48_1024x1024_16k_landcover")

# Trainer Setting

In [ ]:
# 设置样本集
trainer.setup_datasets(train_dataset)
trainer.setup_datasets(val_dataset, data_type="val")
trainer.setup_datasets(test_dataset, data_type="test")

# 设置pretrained model
from aiearth.deeplearning.model_zoo.model import PretrainedModel
model = PretrainedModel("aie://LandCover/landcover_v1.6.pth")
trainer.load_from(model.local_path)

# 设置训练参数
trainer.cfg.runner["max_iters"] = 2  
trainer.cfg.checkpoint_config["interval"]=1
trainer.cfg.data.samples_per_gpu = 1

In [ ]:
# 训练
trainer.train(validate=False)

In [ ]:
import os
# 保存的训练checkpoint路径
checkpoint = os.path.join(work_dir, "latest.pth")

# Test

In [ ]:
trainer.test(checkpoint, output_dir="test_ret")

# Onnx Export

In [ ]:
onnx_path = checkpoint.replace(".pth", ".onnx")

trainer.export_onnx(output_file=onnx_path, checkpoint_path=checkpoint,shape=(1024, 1024))